<a href="https://colab.research.google.com/github/Shostina/ML/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [33]:
import pandas as pd
%cd /content/drive/My\ Drive/ML
train = pd.read_csv('LondonTrain.csv')
featuresFoursquare = pd.read_csv('featuresLondon/Foursquare/venueCategoriesLDA6Features.csv')
featuresInstagram = pd.read_csv('featuresLondon/Instagram/imageConceptsFeatures.csv')
featuresTwitterLDA50 = pd.read_csv('featuresLondon/Twitter/LDA50Features.csv')
featuresTwitterLIWC = pd.read_csv('featuresLondon/Twitter/LIWCFeatures.csv')
featuresTwitterText = pd.read_csv('featuresLondon/Twitter/manuallyDefinedTextFeatures.csv')

/content/drive/My Drive/ML


In [5]:
income_res = train[["row ID", "income"]].dropna()
gender_res = train[["row ID", "gender"]].dropna()
gender_train = gender_res.join(featuresInstagram.set_index("_id"), on="row ID").dropna()
gender_res = gender_train[["gender"]]
gender_feat = pd.DataFrame(gender_train)
del(gender_feat['row ID'])
del(gender_feat['gender'])
income_train = income_res.join(featuresInstagram.set_index("_id"), on="row ID").dropna()
income_res = income_train[["income"]]
income_feat = pd.DataFrame(income_train)
del(income_feat['row ID'])
del(income_feat['income'])

In [ ]:
train

,row ID,educationInfoForAgeGroupEstimation,workInfoForAgeGroupEstimation,gender,realAge,ageGroup,relationship,educationLevel,occupation,income
0,b91f2c096c71434747b3fc62c324abc3,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN
1,1f8b8ec545b6e0fe8d86da7f1dcbb43c,NaN,Edenred Employee Benefits Advisor � November 2...,female,NaN,NaN,NaN,NaN,personal care and service,NaN
2,ecec708ce2078de02beaec22c86c279f,"Uppsala University Egyptology � Uppsala, Sweden",NaN,male,NaN,NaN,NaN,undergraduate,NaN,NaN
3,e87606f217aa0a10bff85bd4cdce129f,NaN,"Booking.com Cambridge, Cambridgeshire",male,NaN,NaN,NaN,NaN,NaN,NaN
4,8bbc0c87b6b1a31c99ecd11a326142d7,University of Verona 2008 to 2010 � Nursing � ...,Coogee Pavilion Pizza Chef � 13 April 2015 to ...,male,NaN,NaN,single,undergraduate,food preparation and service related,NaN
...,...,...,...,...,...,...,...,...,...,...
3273,c30ad48a872006252602aa3f7733321a,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN
3274,753b3f95a54291c9d698d3640e8f73b7,Tottenham College of Technology Class of 1979 ...,NaN,female,NaN,NaN,NaN,college,NaN,NaN
3275,c1c3b60af322612f60a72a46572c9bbe,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN
3276,8e481d2afd7803523314992885582a50,The Magna Carta School Class of 1994 � Staines...,Medivet Richmond Head Nurse � 25 November 2013...,male,NaN,NaN,NaN,school,healthcare practitioners and technical,NaN


In [6]:
clf = svm.SVC(kernel='linear', C=1).fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(clf, gender_feat.values, gender_res.values, cv=5)
print("Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(clf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

clf = svm.SVC(kernel='linear', C=1).fit(income_feat.values, income_res.values)
accuracy = cross_val_score(clf, income_feat.values, income_res.values, cv=5)
print("Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(clf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Gender Accuracy: 0.7010019267822736  +/- 0.04256917481681755
Gender F1 Macro: 0.6522215921723069  +/- 0.05499195116962284


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=Tru

Income Accuracy: 0.3810526315789474  +/- 0.07341303483857978
Income F1 Macro: 0.30310305993742526  +/- 0.13718647747554644


In [10]:
rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(rf, gender_feat.values, gender_res.values, cv=5)
print("Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(rf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(rf, income_feat.values, income_res.values, cv=5)
print("Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(rf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

Gender Accuracy: 0.6535611962804724  +/- 0.032389844947290795
Gender F1 Macro: 0.40856404705141314  +/- 0.022723181921450432


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estima

Income Accuracy: 0.3931578947368421  +/- 0.15404427898885656
Income F1 Macro: 0.26697003976415745  +/- 0.11744090750562261


In [12]:
nb = GaussianNB()
nb.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(nb, gender_feat.values, gender_res.values, cv=5)
print("Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

nb.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(nb, income_feat.values, income_res.values, cv=5)
print("Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

Gender Accuracy: 0.5540118957862109  +/- 0.1631340735520915
Gender F1 Macro: 0.46349535500463224  +/- 0.07373120169747216
Income Accuracy: 0.27736842105263154  +/- 0.09234405136615288
Income F1 Macro: 0.18405483405483403  +/- 0.08764544163445807


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/l

In [15]:
income_res = train[["row ID", "income"]].dropna()
gender_res = train[["row ID", "gender"]].dropna()
gender_train = gender_res.join(featuresFoursquare.set_index("_id"), on="row ID").dropna()
gender_res = gender_train[["gender"]]
gender_feat = pd.DataFrame(gender_train)
del(gender_feat['row ID'])
del(gender_feat['gender'])
income_train = income_res.join(featuresFoursquare.set_index("_id"), on="row ID").dropna()
income_res = income_train[["income"]]
income_feat = pd.DataFrame(income_train)
del(income_feat['row ID'])
del(income_feat['income'])

nb = GaussianNB()
nb.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(nb, gender_feat.values, gender_res.values, cv=5)
print("NB Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("NB Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

nb.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(nb, income_feat.values, income_res.values, cv=5)
print("NB Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("NB Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )
rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(rf, gender_feat.values, gender_res.values, cv=5)
print("RF Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(rf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("RF Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(rf, income_feat.values, income_res.values, cv=5)
print("RF Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(rf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("RF Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

clf = svm.SVC(kernel='linear', C=1).fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(clf, gender_feat.values, gender_res.values, cv=5)
print("SVM Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(clf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("SVM Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

clf = svm.SVC(kernel='linear', C=1).fit(income_feat.values, income_res.values)
accuracy = cross_val_score(clf, income_feat.values, income_res.values, cv=5)
print("SVM Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(clf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("SVM Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )



/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

NB Gender Accuracy: 0.675807514519379  +/- 0.02067796653106638
NB Gender F1 Macro: 0.522110691684454  +/- 0.05484164866189599
NB Income Accuracy: 0.3960919540229885  +/- 0.14937647301692578
NB Income F1 Macro: 0.3506766264095486  +/- 0.14467371600381146


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

RF Gender Accuracy: 0.6967400734858362  +/- 0.011784899204207189
RF Gender F1 Macro: 0.4806256025318456  +/- 0.036241942327074346


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

RF Income Accuracy: 0.43034482758620685  +/- 0.16485563813297505
RF Income F1 Macro: 0.2982138577653283  +/- 0.1732996246303244


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

SVM Gender Accuracy: 0.70258622733199  +/- 0.0011248397433847803
SVM Gender F1 Macro: 0.4126581707854068  +/- 0.0003881681751517011
SVM Income Accuracy: 0.4698850574712644  +/- 0.012873563218390815
SVM Income F1 Macro: 0.159830866807611  +/- 0.0029598308668076223


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [16]:
income_res = train[["row ID", "income"]].dropna()
gender_res = train[["row ID", "gender"]].dropna()
gender_train = gender_res.join(featuresTwitterLDA50.set_index("_id"), on="row ID").dropna()
gender_res = gender_train[["gender"]]
gender_feat = pd.DataFrame(gender_train)
del(gender_feat['row ID'])
del(gender_feat['gender'])
income_train = income_res.join(featuresTwitterLDA50.set_index("_id"), on="row ID").dropna()
income_res = income_train[["income"]]
income_feat = pd.DataFrame(income_train)
del(income_feat['row ID'])
del(income_feat['income'])

nb = GaussianNB()
nb.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(nb, gender_feat.values, gender_res.values, cv=5)
print("NB Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("NB Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

nb.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(nb, income_feat.values, income_res.values, cv=5)
print("NB Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("NB Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )
rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(rf, gender_feat.values, gender_res.values, cv=5)
print("RF Gender Accuracy:", accuracy.mean())
f1 = cross_val_score(rf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("RF Gender F1 Macro:", f1.mean())

rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(rf, income_feat.values, income_res.values, cv=5)
print("RF Income Accuracy:", accuracy.mean())
f1 = cross_val_score(rf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("RF Income F1 Macro:", f1.mean())

clf = svm.SVC(kernel='linear', C=1).fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(clf, gender_feat.values, gender_res.values, cv=5)
print("SVM Gender Accuracy:", accuracy.mean())
f1 = cross_val_score(clf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("SVM Gender F1 Macro:", f1.mean())

clf = svm.SVC(kernel='linear', C=1).fit(income_feat.values, income_res.values)
accuracy = cross_val_score(clf, income_feat.values, income_res.values, cv=5)
print("SVM Income Accuracy:", accuracy.mean())
f1 = cross_val_score(clf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("SVM Income F1 Macro:", f1.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

NB Gender Accuracy: 0.5893521393860377  +/- 0.1013817183769566
NB Gender F1 Macro: 0.5777020421854553  +/- 0.08888909411000684
NB Income Accuracy: 0.33563218390804594  +/- 0.0736063092185109
NB Income F1 Macro: 0.29519607267891024  +/- 0.19284969199243093


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

RF Gender Accuracy: 0.7458480502548299
RF Gender F1 Macro: 0.6117315949345026


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

RF Income Accuracy: 0.3425287356321839
RF Income F1 Macro: 0.1929250091747595


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

SVM Gender Accuracy: 0.7162673936233258
SVM Gender F1 Macro: 0.5280201224615576
SVM Income Accuracy: 0.47011494252873565
SVM Income F1 Macro: 0.25963455149501663


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
income_res = train[["row ID", "income"]].dropna()
gender_res = train[["row ID", "gender"]].dropna()
gender_train = gender_res.join(featuresTwitterLIWC.set_index("_id"), on="row ID").dropna()
gender_res = gender_train[["gender"]]
gender_feat = pd.DataFrame(gender_train)
del(gender_feat['row ID'])
del(gender_feat['gender'])
income_train = income_res.join(featuresTwitterLIWC.set_index("_id"), on="row ID").dropna()
income_res = income_train[["income"]]
income_feat = pd.DataFrame(income_train)
del(income_feat['row ID'])
del(income_feat['income'])

nb = GaussianNB()
nb.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(nb, gender_feat.values, gender_res.values, cv=5)
print("NB Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("NB Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

nb.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(nb, income_feat.values, income_res.values, cv=5)
print("NB Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("NB Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )
rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(rf, gender_feat.values, gender_res.values, cv=5)
print("RF Gender Accuracy:", accuracy.mean())
f1 = cross_val_score(rf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("RF Gender F1 Macro:", f1.mean())

rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(rf, income_feat.values, income_res.values, cv=5)
print("RF Income Accuracy:", accuracy.mean())
f1 = cross_val_score(rf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("RF Income F1 Macro:", f1.mean())

In [ ]:
income_res = train[["row ID", "income"]].dropna()
gender_res = train[["row ID", "gender"]].dropna()
gender_train = gender_res.join(featuresTwitterText.set_index("_id"), on="row ID").dropna()
gender_res = gender_train[["gender"]]
gender_feat = pd.DataFrame(gender_train)
del(gender_feat['row ID'])
del(gender_feat['gender'])
income_train = income_res.join(featuresTwitterText.set_index("_id"), on="row ID").dropna()
income_res = income_train[["income"]]
income_feat = pd.DataFrame(income_train)
del(income_feat['row ID'])
del(income_feat['income'])

nb = GaussianNB()
nb.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(nb, gender_feat.values, gender_res.values, cv=5)
print("NB Gender Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("NB Gender F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )

nb.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(nb, income_feat.values, income_res.values, cv=5)
print("NB Income Accuracy:", accuracy.mean(), " +/-", accuracy.std() * 2)
f1 = cross_val_score(nb, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("NB Income F1 Macro:", f1.mean(), " +/-", f1.std() * 2 )
rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(gender_feat.values, gender_res.values)
accuracy = cross_val_score(rf, gender_feat.values, gender_res.values, cv=5)
print("RF Gender Accuracy:", accuracy.mean())
f1 = cross_val_score(rf, gender_feat.values, np.ravel(gender_res.values), cv=5, scoring='f1_macro')
print("RF Gender F1 Macro:", f1.mean())

rf = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)
rf.fit(income_feat.values, income_res.values)
accuracy = cross_val_score(rf, income_feat.values, income_res.values, cv=5)
print("RF Income Accuracy:", accuracy.mean())
f1 = cross_val_score(rf, income_feat.values, np.ravel(income_res.values), cv=5, scoring='f1_macro')
print("RF Income F1 Macro:", f1.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

NB Gender Accuracy: 0.6832512832255233  +/- 0.011520235291983526
NB Gender F1 Macro: 0.5276910631641676  +/- 0.06063844672594926
NB Income Accuracy: 0.2613793103448276  +/- 0.0947014835518448
NB Income F1 Macro: 0.2610096410423878  +/- 0.11531010964951496


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

RF Gender Accuracy: 0.7061222737420574
RF Gender F1 Macro: 0.5032714382417947


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

RF Income Accuracy: 0.3491954022988505
RF Income F1 Macro: 0.18891237147480283


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
def load_data(features):
  income_res = train[["row ID", "income"]].dropna()
  gender_res = train[["row ID", "gender"]].dropna()
  gender_train = gender_res.join(features.set_index("_id"), on="row ID").dropna()
  gender_res = gender_train[["gender"]]
  gender_feat = pd.DataFrame(gender_train)
  gender_row_id = gender_feat['row ID']
  del(gender_feat['row ID'])
  del(gender_feat['gender'])
  income_train = income_res.join(features.set_index("_id"), on="row ID").dropna()
  income_res = income_train[["income"]]
  income_feat = pd.DataFrame(income_train)
  income_row_id = income_feat['row ID']
  del(income_feat['row ID'])
  del(income_feat['income'])
  return(gender_feat, gender_res, income_feat, income_res, gender_row_id, income_row_id)

In [69]:
(gender_feat_i, gender_res_i, income_feat_i, income_res_i, g_i_rows, i_i_rows) = load_data(featuresInstagram)
g_clf_i = svm.SVC(kernel='linear', C=1, probability = True).fit(gender_feat_i.values, gender_res_i.values)
i_clf_i = (RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)).fit(income_feat_i.values, income_res_i.values)
(gender_feat_f, gender_res_f, income_feat_f, income_res_f, g_f_rows, i_f_rows) = load_data(featuresFoursquare)
g_clf_f = svm.SVC(kernel='linear', C=1, probability=True).fit(gender_feat_f.values, gender_res_f.values)
i_clf_f = svm.SVC(kernel='linear', C=1).fit(income_feat_f.values, income_res_f.values)
(gender_feat_t_lda50, gender_res_t_lda50, income_feat_t_lda50, income_res_t_lda50, g_lda50_rows, i_lda50_rows) = load_data(featuresTwitterLDA50)
g_clf_t_lda50 = (RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)).fit(gender_feat_t_lda50.values, gender_res_t_lda50.values)
i_clf_t_lda50 = svm.SVC(kernel='linear', C=1, probability=True).fit(income_feat_t_lda50.values, income_res_t_lda50.values)
(gender_feat_t_liwc, gender_res_t_liwc, income_feat_t_liwc, income_res_t_liwc, g_liwc_rows, i_liwc_rows) = load_data(featuresTwitterLIWC)
g_clf_t_liwc = (RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)).fit(gender_feat_t_liwc.values, gender_res_t_liwc.values)
i_clf_t_liwc = (RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)).fit(income_feat_t_liwc.values, income_res_t_liwc.values)
(gender_feat_t_text, gender_res_t_text, income_feat_t_text, income_res_t_text, g_text_rows, i_text_rows) = load_data(featuresTwitterText)
g_clf_t_text = (RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)).fit(gender_feat_t_text.values, gender_res_t_text.values)
i_clf_t_text = (RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              max_features='sqrt',
                              random_state=1)).fit(income_feat_t_text.values, income_res_t_text.values)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=T

In [73]:
gender_test_rows = pd.concat([g_i_rows, g_f_rows, g_lda50_rows, g_liwc_rows, g_text_rows], axis=1, join='inner')
gender_test_rows = gender_test_rows.values
g_i_test_feat = []
g_f_test_feat = []
g_Lda_test_feat = []
g_LIWC_test_feat = []
g_Text_test_feat = []
for (id, id1, id2, id3, id4) in gender_test_rows:
  a_i = featuresInstagram.loc[featuresInstagram['_id'].str.contains(id)] 
  del(a_i["_id"])
  g_i_test_feat.append(a_i.values[0])
  a_f = featuresFoursquare.loc[featuresFoursquare['_id'].str.contains(id)] 
  del(a_f["_id"])
  g_f_test_feat.append(np.array(a_f.values[0]))
  a_LIWC = featuresTwitterLIWC.loc[featuresTwitterLIWC['_id'].str.contains(id)] 
  del(a_LIWC["_id"])
  g_LIWC_test_feat.append(np.array(a_LIWC.values[0]))
  a_Text = featuresTwitterText.loc[featuresTwitterText['_id'].str.contains(id)] 
  del(a_Text["_id"])
  g_Text_test_feat.append(np.array(a_Text.values[0]))
  a_Lda = featuresTwitterLDA50.loc[featuresTwitterLDA50['_id'].str.contains(id)] 
  del(a_Lda["_id"])
  g_Lda_test_feat.append(np.array(a_Lda.values[0]))


In [98]:
g_clf_i_res = g_clf_i.predict_proba(g_i_test_feat)
g_clf_f_res = g_clf_f.predict_proba(g_f_test_feat)
g_clf_t_lda50_res = g_clf_t_lda50.predict_proba(g_Lda_test_feat)
g_clf_t_liwc_res = g_clf_t_liwc.predict_proba(g_LIWC_test_feat)
g_clf_t_text_res = g_clf_t_text.predict_proba(g_Text_test_feat)

In [89]:
train_feat_2 = []
for i in range(len(g_clf_i_res)):
  train_feat_2.append(np.concatenate((g_clf_i_res[i], g_clf_f_res[i], g_clf_t_lda50_res[i], g_clf_t_liwc_res[i], g_clf_t_text_res[i])))

In [94]:
train_res_2 = []
for (id, id1, id2, id3, id4) in gender_test_rows:
  a = train.loc[train['row ID'].str.contains(id)] 
  a = a[["gender"]]
  train_res_2.append(a.values[0])

In [97]:
clf = svm.SVC(kernel='linear', C=1, probability = True)
accuracy = cross_val_score(clf, train_feat_2, train_res_2, cv=5)
print("2 Gender Accuracy:", accuracy.mean())
f1 = cross_val_score(clf, train_feat_2, train_res_2, cv=5, scoring='f1_macro')
print("2 Gender F1 Macro:", f1.mean())

2 Gender Accuracy: 0.9987692307692309
2 Gender F1 Macro: 0.9986257928118393


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [137]:
def my_merge(a, ids):
  for i in ids:
    ids[i].append(0.5)
    ids[i].append(0.5)
  for i in a:
    ids[i][-1] = a[i][-1]
    ids[i][-2] = a[i][-2]
  return ids

In [ ]:
g_clf_i_res = g_clf_i.predict_proba(gender_feat_i)
g_clf_f_res = g_clf_f.predict_proba(gender_feat_f)
g_clf_t_lda50_res = g_clf_t_lda50.predict_proba(gender_feat_t_lda50)
g_clf_t_liwc_res = g_clf_t_liwc.predict_proba(gender_feat_t_liwc)
g_clf_t_text_res = g_clf_t_text.predict_proba(gender_feat_t_text)

In [144]:
g_clf_i_res_2 = dict()
g_clf_f_res_2 = dict()
g_clf_lda50_res_2 = dict()
g_clf_liwc_res_2 = dict()
g_clf_text_res_2 = dict()
res = dict()
for i in range(len(g_i_rows.values)):
  a = g_clf_i_res[i].tolist()
  id = (str(g_i_rows.values[i]))
  g_clf_i_res_2[id] = a
  if (id not in res):
    res[id] = []
for i in range(len(g_f_rows.values)):
  a = g_clf_f_res[i].tolist()
  id = (str(g_f_rows.values[i]))
  g_clf_f_res_2[id] = a
  if (id not in res):
    res[id] = []
for i in range(len(g_lda50_rows.values)):
  a = g_clf_t_lda50_res[i].tolist()
  id = (str(g_lda50_rows.values[i]))
  g_clf_lda50_res_2[id] = a
  if (id not in res):
    res[id] = []
for i in range(len(g_liwc_rows.values)):
  a = g_clf_t_liwc_res[i].tolist()
  id = (str(g_liwc_rows.values[i]))
  g_clf_liwc_res_2[id] = a
  if (id not in res):
    res[id] = []
for i in range(len(g_text_rows.values)):
  a = g_clf_t_text_res[i].tolist()
  id = (str(g_text_rows.values[i]))
  g_clf_text_res_2[id] = a
  if (id not in res):
    res[id] = []
res_2 = my_merge(g_clf_i_res_2, res)
res_2 = my_merge(g_clf_f_res_2, res_2)
res_2 = my_merge(g_clf_lda50_res_2, res_2)
res_2 = my_merge(g_clf_liwc_res_2, res_2)
res_2 = my_merge(g_clf_text_res_2, res_2)


In [140]:
keys = res_2.keys()
train_feat_2 = []
for k in keys:
  train_feat_2.append(res[k])
train_res_2 = []
for id in keys:
  a = train.loc[train['row ID'].str.contains(id)] 
  a = a[["gender"]]
  train_res_2.append(a.values[0])

In [143]:

clf = svm.SVC(kernel='linear', C=1, probability = True)
accuracy = cross_val_score(clf, train_feat_2, train_res_2, cv=5)
print("2 Gender Accuracy:", accuracy.mean())
f1 = cross_val_score(clf, train_feat_2, train_res_2, cv=5, scoring='f1_macro')
print("2 Gender F1 Macro:", f1.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

2 Gender Accuracy: 0.9898328789854214


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

2 Gender F1 Macro: 0.9875267835988513


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
